### Current Idea (open to other ideas of course!):


Traditional factor models can struggle capturing short-horizon "micro-regimes" in stock price behavior. The Goal of this project is to use a small autoencoder to learn regimes from 20-day *(I just picked a number we can change this obviously)* sliding windows of price/volume features, and then studiy:

- What the regimes look like
- How they transition from one to another
- Whether transitions have predictive power for near-term returns or volatitlity
- Potential comparissons to simple baselines (momentum, volatility)

### Gathering Data 


####  Optional Download (You shouldn't need to do this) Please use the link in the readme to get the dataset

In [1]:
# uncomment to download data. 
# Not needed if you have the .parquet file in the data/historical folder

# !python qualifier\utils\download_stock_data.py

### Load Dataset from file

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Set random seeds for reproducibility
np.random.seed(42)

# Data path
PARQUET_PATH = Path("data/historical/all_stocks_historical.parquet")

print(f"Loading data from: {PARQUET_PATH}")
df = pd.read_parquet(PARQUET_PATH)
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn dtypes:")
print(df.dtypes)

Loading data from: data\historical\all_stocks_historical.parquet

DataFrame shape: (89859344, 7)

Column dtypes:
ticker            object
date      datetime64[us]
open             float64
high             float64
low              float64
close            float64
volume           float64
dtype: object


In [3]:
# standardize and prepare data
print("Available columns:", df.columns.tolist())

# ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Sort by ticker and date
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

print("\n" + "=" * 60)
print("Data Summary:")
print("=" * 60)
print(f"Total rows: {len(df):,}")
print(f"Unique tickers: {df['ticker'].nunique():,}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Days covered: {(df['date'].max() - df['date'].min()).days:,} days")
print(f"Date column type: {df['date'].dtype}")

Available columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']

Data Summary:
Total rows: 89,859,344
Unique tickers: 7,144
Date range: 1962-01-02 to 2025-11-13
Days covered: 23,326 days
Date column type: datetime64[us]


### Cleaning Data

#### Handling Nans

In [4]:
# original data with nans
print("\n" + "=" * 60)
print("Original Data:")
print("=" * 60)
display(df.head())

# no nan 
print("\n" + "=" * 60)
print("valid (non-null) data:")
print("=" * 60)

# remove rows with nans
cleaned_df = df.dropna(subset=['close', 'volume'])
display(cleaned_df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']].head())


Original Data:


,ticker,date,open,high,low,close,volume
0,A,1962-01-02,NaN,NaN,NaN,NaN,NaN
1,A,1962-01-03,NaN,NaN,NaN,NaN,NaN
2,A,1962-01-04,NaN,NaN,NaN,NaN,NaN
3,A,1962-01-05,NaN,NaN,NaN,NaN,NaN
4,A,1962-01-08,NaN,NaN,NaN,NaN,NaN



valid (non-null) data:


,ticker,date,open,high,low,close,volume
9539,A,1999-11-18,27.245214,29.939798,23.951837,26.347021,62546380.0
9540,A,1999-11-19,25.710805,25.748231,23.839568,24.176392,15234146.0
9541,A,1999-11-22,24.737756,26.347021,23.989261,26.347021,6577870.0
9542,A,1999-11-23,25.448832,26.122478,23.951841,23.951841,5975611.0
9543,A,1999-11-24,24.026691,25.112009,23.951841,24.588062,4843231.0


#### Trimming dates

In [5]:
# trimming the df_valid to start at the year 2000
cleaned_trimmed_df = cleaned_df[cleaned_df['date'] >= '2000-01-01'].copy()
print("\n" + "=" * 60)
print("Filtered data (Year 2000 onwards):")
print("=" * 60)
print(f"Total rows: {len(cleaned_trimmed_df):,}")
print(f"Date range: {cleaned_trimmed_df['date'].min().date()} to {cleaned_trimmed_df['date'].max().date()}")
display(cleaned_trimmed_df[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']].head())


Filtered data (Year 2000 onwards):
Total rows: 21,521,135
Date range: 2000-01-03 to 2025-11-13


,ticker,date,open,high,low,close,volume
9569,A,2000-01-03,47.155195,47.267470,40.343889,43.113323,4674353.0
9570,A,2000-01-04,40.792992,41.242092,38.772056,39.819950,4765083.0
9571,A,2000-01-05,39.670253,39.819954,36.114903,37.349918,5758642.0
9572,A,2000-01-06,36.900802,37.125352,34.805015,35.927757,2534434.0
9573,A,2000-01-07,35.366397,39.483121,35.328974,38.921749,2819626.0


#### Removing stocks that do not have at least 1000 days of data and Filtering for highest volume stocks

In [6]:
# two-stage universe selection/filtering to shrink for MVP
# 1. remove tickers with insufficient history
# 2. select top N by average volume from quality-filtered set


min_days_required = 1000  # min number of days per ticker
top_n = 50  # target universe size

print("\n" + "=" * 60)
print(f"Stage 1: Quality Filter (min {min_days_required} trading days)")
print("=" * 60)

# count days per ticker in the full trimmed dataset
rows_per_ticker_all = cleaned_trimmed_df.groupby('ticker').size()
valid_tickers = rows_per_ticker_all[rows_per_ticker_all >= min_days_required].index.tolist()

print(f"Tickers before quality filter: {len(rows_per_ticker_all):,}")
print(f"Tickers after quality filter:  {len(valid_tickers):,}")
print(f"Tickers removed: {len(rows_per_ticker_all) - len(valid_tickers):,}")

# filter to quality tickers only
quality_filtered_df = cleaned_trimmed_df[cleaned_trimmed_df['ticker'].isin(valid_tickers)].copy()

print("\n" + "=" * 60)
print(f"Stage 2: Liquidity Ranking (top {top_n} by avg volume)")
print("=" * 60)

# average volume on quality-filtered stocks
avg_volume_by_ticker = quality_filtered_df.groupby('ticker')['volume'].mean().sort_values(ascending=False)

# select top N from the filtered universe
universe = avg_volume_by_ticker.head(top_n).index.tolist()



Stage 1: Quality Filter (min 1000 trading days)
Tickers before quality filter: 7,093
Tickers after quality filter:  5,182
Tickers removed: 1,911

Stage 2: Liquidity Ranking (top 50 by avg volume)


***ONLY USE 'data' IN THE NEXT STEPS. IT IS CLEANED AND TRIMMED***

In [7]:
# create FINAL FINAL dataset
data = quality_filtered_df[quality_filtered_df['ticker'].isin(universe)].copy()
data = data.sort_values(['ticker', 'date']).reset_index(drop=True)

print(f"Selected universe size: {len(universe)} tickers (target: {top_n})")
print(f"{len(universe)} tickers have >= {min_days_required} days of history")

print("-" * 60)
for i, (ticker, vol) in enumerate(avg_volume_by_ticker.head(10).items(), 1):
    print(f"  {i:2d}. {ticker:6s} - {vol:>15,.0f} shares/day")

Selected universe size: 50 tickers (target: 50)
50 tickers have >= 1000 days of history
------------------------------------------------------------
   1. NVDA   -     600,617,217 shares/day
   2. AAPL   -     374,911,010 shares/day
   3. AMZN   -     116,037,570 shares/day
   4. GOOGL  -     112,346,485 shares/day
   5. GOOG   -     110,036,509 shares/day
   6. TSLA   -      96,896,489 shares/day
   7. TQQQ   -      84,554,877 shares/day
   8. BAC    -      77,327,168 shares/day
   9. QQQ    -      66,639,305 shares/day
  10. NIO    -      61,121,488 shares/day


In [8]:
print("\n" + "=" * 60)
print(f"Final Universe: {len(universe)} stocks")
print("=" * 60)
print(f"Total rows: {len(data):,}")
print(f"Unique tickers: {data['ticker'].nunique()}")
print(f"Date range: {data['date'].min().date()} to {data['date'].max().date()}")


print("\n" + "-" * 60)
print("Data Completeness (per ticker):")
print("-" * 60)
rows_per_ticker = data.groupby('ticker').size()
print(f"  Min:    {rows_per_ticker.min():>5,} days  (>= {min_days_required} guaranteed)")
print(f"  Median: {rows_per_ticker.median():>5,.0f} days")
print(f"  Mean:   {rows_per_ticker.mean():>5,.0f} days")
print(f"  Max:    {rows_per_ticker.max():>5,} days")

print("\n" + "=" * 60)
print("Selected Tickers (sorted by avg volume):")
print(f"  {', '.join(universe)}")

display(data.head(10))


Final Universe: 50 stocks
Total rows: 244,098
Unique tickers: 50
Date range: 2000-01-03 to 2025-11-13

------------------------------------------------------------
Data Completeness (per ticker):
------------------------------------------------------------
  Min:    1,007 days  (>= 1000 guaranteed)
  Median: 6,506 days
  Mean:   4,882 days
  Max:    6,507 days

Selected Tickers (sorted by avg volume):
  NVDA, AAPL, AMZN, GOOGL, GOOG, TSLA, TQQQ, BAC, QQQ, NIO, PLTR, INTC, MSFT, F, SOFI, CSCO, OPEN, AMD, PFE, RIVN, T, SNAP, CMG, WMT, META, EBAY, ORCL, AVGO, ITUB, CMCSA, CSX, LRCX, HPQ, UBER, WFC, VALE, MU, BBAI, TLRY, GRAB, NOK, RGTI, HOOD, JPM, BABA, C, AMAT, PBR, XOM, BBD


,ticker,date,open,high,low,close,volume
0,AAPL,2000-01-03,0.786328,0.843498,0.762428,0.839281,5.357968e+08
1,AAPL,2000-01-04,0.811633,0.829440,0.758680,0.768521,5.123776e+08
2,AAPL,2000-01-05,0.777892,0.828971,0.772269,0.779767,7.783216e+08
3,AAPL,2000-01-06,0.795700,0.802260,0.712287,0.712287,7.679728e+08
4,AAPL,2000-01-07,0.723534,0.757274,0.716037,0.746027,4.607344e+08
5,AAPL,2000-01-10,0.764771,0.766646,0.710413,0.732906,5.050640e+08
6,AAPL,2000-01-11,0.719316,0.745090,0.678547,0.695417,4.415488e+08
7,AAPL,2000-01-12,0.712287,0.716036,0.648556,0.653711,9.760688e+08
8,AAPL,2000-01-13,0.708421,0.740403,0.693543,0.725408,1.032685e+09
9,AAPL,2000-01-14,0.749776,0.766646,0.745090,0.753056,3.903760e+08


### Exploring data

#### Feature engineering

In [9]:
feat_df = data.copy()
feat_df = feat_df.sort_values(['ticker', 'date']).reset_index(drop=True)

# 1. log price & 1-day log return 
feat_df['log_price'] = np.log(feat_df['close'])
feat_df['log_ret_1d'] = feat_df.groupby('ticker')['log_price'].diff()

# 2. rolling volatilities of returns 
feat_df['vol_5d'] = (
    feat_df.groupby('ticker')['log_ret_1d']
           .rolling(window=5, min_periods=5)
           .std()
           .reset_index(level=0, drop=True)
)

feat_df['vol_20d'] = (
    feat_df.groupby('ticker')['log_ret_1d']
           .rolling(window=20, min_periods=20)
           .std()
           .reset_index(level=0, drop=True)
)

# 3. vol z-score with 1-period lag to exclude current row
g = feat_df.groupby('ticker')['volume']
exp_mean = g.expanding().mean().shift(1).reset_index(level=0, drop=True)
exp_std  = g.expanding().std(ddof=0).shift(1).reset_index(level=0, drop=True)

feat_df['volume_mean_exp'] = exp_mean
feat_df['volume_std_exp']  = exp_std

# avoid dividing by 0
feat_df['volume_z'] = (
    (feat_df['volume'] - feat_df['volume_mean_exp']) 
    / (feat_df['volume_std_exp'] + 1e-8)
)


# 4. intraday range
feat_df['range_frac'] = (feat_df['high'] - feat_df['low']) / feat_df['close']

# Keep only needed columns
feature_cols = ['log_ret_1d', 'vol_5d', 'vol_20d', 'volume_z', 'range_frac']

feat_df = feat_df[['ticker', 'date'] + feature_cols]

# drop nan rows that were created because of frolling features etc
feat_df = feat_df.dropna(subset=feature_cols).reset_index(drop=True)

print("Feature DF shape:", feat_df.shape)
feat_df.head()

Feature DF shape: (243098, 7)


,ticker,date,log_ret_1d,vol_5d,vol_20d,volume_z,range_frac
0,AAPL,2000-02-01,-0.034317,0.037641,0.054475,-0.927957,0.049875
1,AAPL,2000-02-02,-0.014443,0.037796,0.050977,-0.447783,0.051866
2,AAPL,2000-02-03,0.044535,0.048172,0.051917,-0.401788,0.038718
3,AAPL,2000-02-04,0.044373,0.035449,0.048214,-0.547381,0.059028
4,AAPL,2000-02-07,0.054615,0.040390,0.048611,-0.482604,0.072876


#### Feature Correlation

In [10]:
import plotly.express as px
import plotly.io as pio
corr = feat_df[feature_cols].corr()

fig = px.imshow(
    corr,
    text_auto=".2f",
    color_continuous_scale="RdBu_r",
    origin="lower",
    title="Correlation between features"
)
fig.update_layout(height=500, width=600)
fig.show()


### Modeling 

####  Unsupervised learning

##### Creating windows

In [11]:
window_len = 20 # how big (how many days) the window is


# function to make le windows of 20 days for each ticker
def make_window_features(feat_df, feature_cols, window_len=20):
    
    
    frames = []
    # looping through the tickers and creating windows
    for ticker, g in feat_df.groupby('ticker'):
        g = g.sort_values('date').reset_index(drop=True)

        roll = g[feature_cols].rolling(window_len, min_periods=window_len)

        win_feats = pd.concat(
            [
                roll.mean().add_suffix('_mean'),
                roll.std().add_suffix('_std'),
                roll.min().add_suffix('_min'),
                roll.max().add_suffix('_max'),
            ],
            axis=1,
        )

        tmp = pd.concat([g[['ticker', 'date']], win_feats], axis=1).dropna()
        frames.append(tmp)

    window_df = pd.concat(frames, ignore_index=True)
    return window_df

window_df = make_window_features(
    feat_df,
    feature_cols=['log_ret_1d', 'vol_5d', 'vol_20d', 'volume_z', 'range_frac'],
    window_len=window_len
)

print(window_df.shape)
window_df.head()


(242148, 22)


,ticker,date,log_ret_1d_mean,vol_5d_mean,vol_20d_mean,volume_z_mean,range_frac_mean,log_ret_1d_std,vol_5d_std,vol_20d_std,...,log_ret_1d_min,vol_5d_min,vol_20d_min,volume_z_min,range_frac_min,log_ret_1d_max,vol_5d_max,vol_20d_max,volume_z_max,range_frac_max
0,AAPL,2000-02-29,0.004984,0.035715,0.042681,-0.624599,0.052269,0.033529,0.006877,0.006093,...,-0.042813,0.022928,0.033529,-1.145899,0.020674,0.062921,0.048172,0.054475,-0.150583,0.090061
1,AAPL,2000-03-01,0.013113,0.037051,0.042063,-0.473120,0.054979,0.042111,0.009401,0.005424,...,-0.042813,0.022928,0.033529,-1.145899,0.020674,0.128269,0.064356,0.051917,2.101617,0.104076
2,AAPL,2000-03-02,0.010540,0.038934,0.041781,-0.485537,0.055357,0.045333,0.012739,0.005071,...,-0.065913,0.022928,0.033529,-1.145899,0.020674,0.128269,0.075465,0.051917,2.101617,0.104076
3,AAPL,2000-03-03,0.010714,0.040018,0.041459,-0.497518,0.056638,0.045476,0.014384,0.004573,...,-0.065913,0.022928,0.033529,-1.145899,0.020674,0.128269,0.075465,0.048611,2.101617,0.104076
4,AAPL,2000-03-06,0.007583,0.041904,0.041308,-0.521612,0.055327,0.045191,0.016120,0.004384,...,-0.065913,0.022928,0.033529,-1.145899,0.020674,0.128269,0.075465,0.048611,2.101617,0.104076


##### Creating regimes using k-means

In [12]:
#  window_df we created above is what we'll use for the Unsuper. learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# feature cols in window_df
window_feature_cols = [c for c in window_df.columns if c not in ['ticker', 'date']]

X_win = window_df[window_feature_cols].values

scaler_win = StandardScaler()
X_win_scaled = scaler_win.fit_transform(X_win)

k = 6  # first guess for number of regimes; you can try 4, 6, 8, etc.

kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
window_df['regime_k6'] = kmeans.fit_predict(X_win_scaled)

window_df['regime_k6'].value_counts().sort_index()

regime_k6
0    107841
1     26707
2     68321
3      1240
4     28494
5      9545
Name: count, dtype: int64

In [13]:
# exploring the regime windows
regime_profile = (
    window_df
    .groupby('regime_k6')[window_feature_cols]
    .mean()
    .assign(count=window_df.groupby('regime_k6').size())
)

regime_profile

,log_ret_1d_mean,vol_5d_mean,vol_20d_mean,volume_z_mean,range_frac_mean,log_ret_1d_std,vol_5d_std,vol_20d_std,volume_z_std,range_frac_std,...,vol_5d_min,vol_20d_min,volume_z_min,range_frac_min,log_ret_1d_max,vol_5d_max,vol_20d_max,volume_z_max,range_frac_max,count
regime_k6,,,,,,,,,,,,,,,,,,,,,
0,0.000805,0.012012,0.013342,-0.337576,0.017881,0.012781,0.004410,0.001800,0.386532,0.006806,...,0.005291,0.010528,-0.850383,0.008866,0.026032,0.020004,0.016071,0.696571,0.035012,107841
1,0.000023,0.022587,0.023717,1.548629,0.031494,0.024552,0.008743,0.003184,1.270830,0.012915,...,0.009818,0.018614,0.150027,0.015026,0.048542,0.038343,0.028425,5.273126,0.064819,26707
2,0.000632,0.022801,0.024656,-0.102130,0.030789,0.024391,0.008815,0.003395,0.511532,0.012236,...,0.009986,0.019266,-0.750020,0.014943,0.050189,0.038879,0.029640,1.291660,0.061974,68321
3,0.001757,0.112592,0.113157,3.968209,0.145015,0.130420,0.061172,0.024528,4.320647,0.084268,...,0.040431,0.075061,0.893097,0.056425,0.289389,0.226473,0.145379,18.945347,0.392013,1240
4,-0.000504,0.040106,0.042881,0.478864,0.053652,0.042944,0.015727,0.006203,0.785523,0.021434,...,0.017825,0.033154,-0.448950,0.025819,0.087480,0.068921,0.051902,2.690384,0.107802,28494
5,-0.002407,0.064869,0.067437,1.180456,0.086684,0.070604,0.028220,0.011481,1.185722,0.038135,...,0.027187,0.049431,-0.107780,0.039276,0.147106,0.117291,0.083366,4.591741,0.184879,9545


#### Supervised learning

##### Creating Future Targets

In [14]:
##### Creating Future Targets

# using feat_df so we can line it by ticker and date
feat_with_target = feat_df.copy()

# number of days ahead
horizon = 5

def add_targets(g, horizon=horizon):
    g = g.sort_values('date').reset_index(drop=True)

    # cumu sum of 1D log returns
    g['cum_log_ret'] = g['log_ret_1d'].cumsum()

    # future horizon log return: sum_{t+1..t+horizon} log_ret_1d
    # = cum_log_ret at t+horizon minus cum_log_ret at t
    g[f'ret_{horizon}d_fwd'] = g['cum_log_ret'].shift(-horizon) - g['cum_log_ret']

    return g

feat_with_target = (
    feat_with_target
    .groupby('ticker', group_keys=False)
    .apply(add_targets, horizon=horizon)
)

# binary label: will the horizon return be positive (CLASSIFICXAION)
feat_with_target[f'up_{horizon}d'] = (feat_with_target[f'ret_{horizon}d_fwd'] > 0).astype(int)

# drop helper column 
feat_with_target = (
    feat_with_target
    .drop(columns=['cum_log_ret'])
    .dropna(subset=[f'ret_{horizon}d_fwd'])
    .reset_index(drop=True)
)

feat_with_target.head()


,ticker,date,log_ret_1d,vol_5d,vol_20d,volume_z,range_frac,ret_5d_fwd,up_5d
0,AAPL,2000-02-01,-0.034317,0.037641,0.054475,-0.927957,0.049875,0.136179,1
1,AAPL,2000-02-02,-0.014443,0.037796,0.050977,-0.447783,0.051866,0.130840,1
2,AAPL,2000-02-03,0.044535,0.048172,0.051917,-0.401788,0.038718,0.094045,1
3,AAPL,2000-02-04,0.044373,0.035449,0.048214,-0.547381,0.059028,0.006920,1
4,AAPL,2000-02-07,0.054615,0.040390,0.048611,-0.482604,0.072876,0.015226,1


##### Merging windows with targets

In [15]:
#### Merging window features, regimes, and future targets

#  window features, regimes, and future targets combined
supervised_df = (
    window_df
    .merge(
        feat_with_target[['ticker', 'date', 'ret_5d_fwd', 'up_5d']],
        on=['ticker', 'date'],
        how='inner'
    )
    .sort_values(['ticker', 'date'])
    .reset_index(drop=True)
)

print(supervised_df.shape)
supervised_df.head()

(241898, 25)


,ticker,date,log_ret_1d_mean,vol_5d_mean,vol_20d_mean,volume_z_mean,range_frac_mean,log_ret_1d_std,vol_5d_std,vol_20d_std,...,volume_z_min,range_frac_min,log_ret_1d_max,vol_5d_max,vol_20d_max,volume_z_max,range_frac_max,regime_k6,ret_5d_fwd,up_5d
0,AAPL,2000-02-29,0.004984,0.035715,0.042681,-0.624599,0.052269,0.033529,0.006877,0.006093,...,-1.145899,0.020674,0.062921,0.048172,0.054475,-0.150583,0.090061,4,0.069501,1
1,AAPL,2000-03-01,0.013113,0.037051,0.042063,-0.473120,0.054979,0.042111,0.009401,0.005424,...,-1.145899,0.020674,0.128269,0.064356,0.051917,2.101617,0.104076,4,-0.065913,0
2,AAPL,2000-03-02,0.010540,0.038934,0.041781,-0.485537,0.055357,0.045333,0.012739,0.005071,...,-1.145899,0.020674,0.128269,0.075465,0.051917,2.101617,0.104076,4,0.002047,1
3,AAPL,2000-03-03,0.010714,0.040018,0.041459,-0.497518,0.056638,0.045476,0.014384,0.004573,...,-1.145899,0.020674,0.128269,0.075465,0.048611,2.101617,0.104076,4,-0.017734,0
4,AAPL,2000-03-06,0.007583,0.041904,0.041308,-0.521612,0.055327,0.045191,0.016120,0.004384,...,-1.145899,0.020674,0.128269,0.075465,0.048611,2.101617,0.104076,4,-0.035429,0


##### Adding regime transition features

In [19]:
from itertools import groupby

# getting the previous regime for each window and each ticker
supervised_df['regime_prev'] = (supervised_df.groupby('ticker')['regime_k6'].shift(1))

# comparing against the previous window to see if there was a regime change
supervised_df['regime_changed'] = (

    # checking that: 
    # prev regime not na 
    # and does not equal current regime
    supervised_df['regime_prev'].notna() & (supervised_df['regime_prev'] != supervised_df['regime_k6']).astype(int)
)

# the first window of the data wont have a regime before it so droping it and reset index
supervised_df = supervised_df.dropna(subset=['regime_prev']).reset_index(drop=True)

supervised_df.head()

,ticker,date,log_ret_1d_mean,vol_5d_mean,vol_20d_mean,volume_z_mean,range_frac_mean,log_ret_1d_std,vol_5d_std,vol_20d_std,...,log_ret_1d_max,vol_5d_max,vol_20d_max,volume_z_max,range_frac_max,regime_k6,ret_5d_fwd,up_5d,regime_prev,regime_changed
0,AAPL,2000-03-01,0.013113,0.037051,0.042063,-0.473120,0.054979,0.042111,0.009401,0.005424,...,0.128269,0.064356,0.051917,2.101617,0.104076,4,-0.065913,0,4.0,False
1,AAPL,2000-03-02,0.010540,0.038934,0.041781,-0.485537,0.055357,0.045333,0.012739,0.005071,...,0.128269,0.075465,0.051917,2.101617,0.104076,4,0.002047,1,4.0,False
2,AAPL,2000-03-03,0.010714,0.040018,0.041459,-0.497518,0.056638,0.045476,0.014384,0.004573,...,0.128269,0.075465,0.048611,2.101617,0.104076,4,-0.017734,0,4.0,False
3,AAPL,2000-03-06,0.007583,0.041904,0.041308,-0.521612,0.055327,0.045191,0.016120,0.004384,...,0.128269,0.075465,0.048611,2.101617,0.104076,4,-0.035429,0,4.0,False
4,AAPL,2000-03-07,0.003721,0.043674,0.041090,-0.536733,0.054252,0.044251,0.017801,0.004101,...,0.128269,0.075797,0.048266,2.101617,0.104076,4,-0.072779,0,4.0,False


##### Supervised feature matrix and targets 

In [ ]:
# Feature matrix = X
# Targets = y

from sklearn.preprocessing import OneHotEncoder

'''
Still working on this one 

-Cale
'''
